In [10]:
import numpy as np 
from qutip import *
from qutip.qip.circuit import *
from qutip.qip.operations import *
import matplotlib as plt 
import random 

# bit flip simulation trial #1 
up = basis(2,0)  # |0> basis state 
# bit setup; state before error 
psi = (basis(2,0) + basis(2, 1)).unit()    # a|0> + b|1> norm. unit vector 
init_state1 = tensor(psi, up , up)

#encoding q1 and q2 (No error)
q1 = cnot(N = 3, control = 0, target = 1)
q2 = cnot(N = 3, control = 0, target = 2)
init_state2 = q2*q1*init_state1            # a|000> + b|111>

# Syndrom meas. 8 x 8 mat
A = tensor(sigmaz(), sigmaz(), qeye(2))  #ZZI
B = tensor(qeye(2), sigmaz(), sigmaz())  #IZZ

#flip operations 
flip0 = tensor(sigmax(), qeye(2), qeye(2))  #q0  |100>
flip1 = tensor(qeye(2), sigmax(), qeye(2))  #q1  |010>
flip2 = tensor(qeye(2), qeye(2), sigmax())  #q2  |001>
no_flip = tensor(qeye(2), qeye(2), qeye(2))  #no flip  |000>

# what qubit gets flipped 
flip = [flip0, flip1, flip2, no_flip]
c = random.choice(flip)
err_state = c*init_state2   # state after error applied 

# Perform syndrome measurement
a = 1 if (A * err_state) == err_state else -1
b = 1 if (B * err_state) == err_state else -1
print(a, b)
'''print(fx)'''
# error detection & correction
if a ==1 and b ==1: 
    print("No flip: \n {}".format(err_state))
elif a == -1 and b == 1:
    print("q0 is flipped: \n {}".format(err_state))
    #err_corr = tensor(sigmax(), qeye(2), qeye(2))*err_state
    err_corr = tensor(sigmax(), qeye(2), qeye(2))*err_state
    print("Corrected to \n {}".format(err_corr))
elif a == -1 and b == -1:
    print("q1 is flipped: \n {}".format(err_state))
    err_corr = tensor(qeye(2), sigmax(), qeye(2))*err_state                 
    print("Corrected to \n {}".format(err_corr))
elif a == 1 and b == -1:
    print("q2 is flipped: \n {}".format(err_state))
    #err_corr = tensor(qeye(2), sigmax(), qeye(2))*err_state
    err_corr = tensor(qeye(2), qeye(2), sigmax())*err_state
    print("Corrected to \n {}".format(err_corr))

-1 -1
q1 is flipped: 
 Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
Qobj data =
[[0.        ]
 [0.        ]
 [0.70710678]
 [0.        ]
 [0.        ]
 [0.70710678]
 [0.        ]
 [0.        ]]
Corrected to 
 Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.70710678]]


In [1]:
'''# Bloch sphere render 
b = qutip.Bloch()
b.make_sphere()
b.add_states(psi)
b.add_states(init_state1)
b.view
b.show()'''

'# Bloch sphere render \nb = qutip.Bloch()\nb.make_sphere()\nb.add_states(psi)\nb.add_states(init_state1)\nb.view\nb.show()'

In [ ]:
# faulty CNOT circuit 
import numpy as np 
from qutip import *
from qutip.qip.circuit import *
from qutip.qip.operations import *
import matplotlib as plt 
import random 

# bit flip simulation trial #1 
up = basis(2,0)  # |0> basis state 
# bit setup; state before error 
psi = (basis(2,0) + basis(2, 1)).unit()    # a|0> + b|1> norm. unit vector 
init_state1 = tensor(psi, up , up)

# faulty cnot construction 
# error in the cnot is caused by a faulty X gate 
err = .9j*np.pi/2*sigmax()
fx = -1j*err.expm()

# Tensor Up
pup = 1/2*(qeye(2) + sigmaz())
pdown = 1/2*(qeye(2) - sigmaz())
#ccnot = tensor([pup, qeye(2)])+tensor([pdown, sigmax()])
# faulty cnot: a rotation added to the cnot to make it slightly inaccurate 
fcnot = tensor([pup, qeye(2)])+tensor([pdown, fx])

# 8 x 8 cnot for a 3 bit system 
fcnot1 = tensor(fcnot, qeye(2))   # caulty orientation for q1
fcnot2 = tensor([pup, qeye(2), qeye(2)]) + tensor([pdown, qeye(2), fx])  # correct orientation for q2 
#ccnot1 = tensor(ccnot, qeye(2))   # correct orientation for q1 
# this matrice will be applied to the system as faulty cnots 
# rotations will be modified to test fault tolerance 

#encoding q1 and q2 (No error)
q1 = cnot(N = 3, control = 0, target = 1)
q2 = cnot(N = 3, control = 0, target = 2)
err_state = q2*fcnot1*init_state1 #init_state2 

# Syndrom meas. 8 x 8 mat
A = tensor(sigmaz(), sigmaz(), qeye(2))  #ZZI
B = tensor(qeye(2), sigmaz(), sigmaz())  #IZZ

'''#flip operations 
flip0 = tensor(sigmax(), qeye(2), qeye(2))  #q0  |100>
flip1 = tensor(qeye(2), sigmax(), qeye(2))  #q1  |010>
flip2 = tensor(qeye(2), qeye(2), sigmax())  #q2  |001>
no_flip = tensor(qeye(2), qeye(2), qeye(2))  #no flip  |000>

# what qubit gets flipped 
flip = [flip0, flip1, flip2, no_flip]
c = random.choice(flip)
err_state = c*init_state2   # state after error applied '''

# Perform syndrome measurement
a = 1 if (A * err_state) == err_state else -1
b = 1 if (B * err_state) == err_state else -1
print(a, b)
'''print(fx)'''
# error detection & correction
if a ==1 and b ==1: 
    print("No flip: \n {}".format(err_state))
elif a == -1 and b == 1:
    print("q0 is flipped: \n {}".format(err_state))
    #err_corr = tensor(sigmax(), qeye(2), qeye(2))*err_state
    err_corr = tensor(sigmax(), qeye(2), qeye(2))*err_state
    print("Corrected to \n {}".format(err_corr))
elif a == -1 and b == -1:
    print("q1 is flipped: \n {}".format(err_state))
    err_corr = tensor(qeye(2), sigmax(), qeye(2))*err_state                 
    print("Corrected to \n {}".format(err_corr))
elif a == 1 and b == -1:
    print("q2 is flipped: \n {}".format(err_state))
    #err_corr = tensor(qeye(2), sigmax(), qeye(2))*err_state
    err_corr = tensor(qeye(2), qeye(2), sigmax())*err_state
    print("Corrected to \n {}".format(err_corr))

In [ ]:
fcnot = tensor([pup, qeye(2), qeye(2)])+tensor([pdown, fx, qeye(2)])
print(fcnot)
c = 2*snot()*fx
print(c)